In [1]:

# Imports + Global Settings

import json
import os
from datetime import datetime
from typing import Dict

# Optional: pretty printing
import textwrap

# Load Ollama local LLM client (try the official package first, fall back to HTTP)
# Preferred: `pip install ollama`
llm = None
try:
    from ollama import Client as OllamaClient
    llm = OllamaClient(host="http://localhost:11434")
    print("🔥 Local LLM client ready (ollama package).")
except Exception as e:
    # Fallback: lightweight HTTP client using requests
    try:
        import requests
        class SimpleOllamaClient:
            def __init__(self, host='http://localhost:11434'):
                self.host = host.rstrip('/')
            def chat(self, model, messages):
                # maps to Ollama's chat endpoint
                url = f"{self.host}/api/chat"
                payload = {"model": model, "messages": messages}
                resp = requests.post(url, json=payload, timeout=30)
                resp.raise_for_status()
                return resp.json()
        llm = SimpleOllamaClient(host="http://localhost:11434")
        print("⚠️ Using fallback HTTP Ollama client (requests).")
    except Exception as e2:
        print("⚠️ Could not import `ollama` package or use HTTP fallback.")
        print("To enable the local LLM client, either:")
        print("  1) pip install ollama")
        print("  2) ensure Ollama daemon is running at http://localhost:11434 and install requests (`pip install requests`)")
        print("Falling back to a stub llm object that raises if used.")
        class _StubLLM:
            def chat(self, *args, **kwargs):
                raise RuntimeError("No Ollama client available. Install `ollama` or `requests` and start the Ollama daemon.")
        llm = _StubLLM()


⚠️ Using fallback HTTP Ollama client (requests).


In [2]:
# ---------------------------------------------------------------
# 📘 Notebook 04 — Cell 2
# Prompt Template + Formatting Function
# ---------------------------------------------------------------

def build_rag_prompt(query: str, context_data: Dict):
    """
    Build the final RAG prompt to send to the local LLM.
    Includes:
    - merged context
    - user question
    - instruction to use citations
    """

    context_block = context_data["merged_context"]

    prompt = f"""
You are a helpful assistant answering questions using ONLY the context provided.

CONTEXT:
-------
{context_block}
-------

INSTRUCTIONS:
- Use ONLY the information in the context.
- Do NOT hallucinate.
- Cite your sources using this format: [source].
- If the answer is not in the context, say: "The answer is not available in the provided context."

USER QUESTION:
{query}

FINAL ANSWER (with citations):
"""

    return prompt.strip()


In [3]:
# ---------------------------------------------------------------
# 📘 Notebook 04 — Cell 3
# Generate Answer using Local LLM (Ollama)
# ---------------------------------------------------------------

def generate_llm_answer(query: str, context_for_llm: Dict, model: str = "llama3"):
    """
    Sends the final prompt to the local LLM and retrieves the response.
    """
    prompt = build_rag_prompt(query, context_for_llm)

    response = llm.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )

    answer = response["message"]["content"]
    return answer


# ---- Test the answering pipeline (only if variables are defined) ----
# If 'context_for_llm' comes from notebook 03, use it. Otherwise, use a sample query.
if 'context_for_llm' in locals():
    # Use the query and context from notebook 03
    sample_answer = generate_llm_answer(query, context_for_llm, model="llama3")
    
    print("\n===== LLM ANSWER =====\n")
    print(sample_answer)
else:
    print("⚠️ Variable 'context_for_llm' not found in kernel.")
    print("Please run Notebook 03 first to generate the retrieval context, then return here.")

⚠️ Variable 'context_for_llm' not found in kernel.
Please run Notebook 03 first to generate the retrieval context, then return here.


In [4]:
# ---------------------------------------------------------------
# 📘 Notebook 04 — Cell 4
# Final Output Formatting (Answer + Citations)
# ---------------------------------------------------------------

def format_final_output(answer: str, context_for_llm: Dict):
    """
    Returns a clean, structured output including:
    - Answer
    - Citations
    """
    print("\n=======================")
    print("📘 FINAL ANSWER")
    print("=======================\n")
    print(answer)

    print("\n=======================")
    print("🔗 CITATIONS")
    print("=======================\n")
    for c in context_for_llm["citations"]:
        print(f"- Source: {c['source']}")
        print(f"  Preview: {c['chunk_preview']}")
        print(f"  Score: {c['rerank_score']:.4f}\n")


# Display final result (only if variables are defined)
if 'sample_answer' in locals() and 'context_for_llm' in locals():
    format_final_output(sample_answer, context_for_llm)
else:
    print("⚠️ Variables 'sample_answer' and/or 'context_for_llm' not found.")
    print("Please run cells above first to generate these variables.")

⚠️ Variables 'sample_answer' and/or 'context_for_llm' not found.
Please run cells above first to generate these variables.
